<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/entrega_final/Proyecto_decoder_bookcorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Carlos Leonardo Ancasi Hinostroza'
print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


#  Decoder

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 33.3 MB/s 
     |████████████████████████████████| 596 kB 55.6 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -q git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

## Importação dos pacotes

In [4]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from pycocotools.coco import COCO

In [5]:
# Check which GPU we are using
!nvidia-smi

Mon Jul 18 19:42:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Download BookCorpus

## Implementação do MyDataset

In [7]:
from typing import List

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return (tokenizer.batch_encode_plus([text], return_tensors=None, add_special_tokens=False).input_ids)[0]


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.x = []
        self.max_seq_length = max_seq_length
        x = [101]
        x.extend([tokenizer.pad_token_id]*self.max_seq_length)        

        for texto in texts:
            token = tokenize(texto, tokenizer)
            for i in range(0, len(token), (self.max_seq_length - 1) ):
                context_size = (self.max_seq_length - 1)
                if i +  max_seq_length - 1 > len(token):
                    context_size = len(token) % (self.max_seq_length - 1)
                x_a = x[:]
                x_a[1:context_size+1]=token[i:i+context_size]

                self.x.append(x_a)           

    def __len__(self):
        # Escreva aqui seu código.
        return len(self.x)

    def __getitem__(self, idx):
        # Escreva aqui seu código.
        return torch.LongTensor(self.x[idx][:-1]), torch.LongTensor(self.x[idx][1:])

## Testando se a implementação do MyDataset está correta

In [8]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=10)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)

#assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))
correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

print(first_batch_input)
print(first_batch_target)

# assert torch.equal(first_batch_input, correct_first_batch_input)
# assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Passou no assert de tamanho do dataset.
tensor([[  101,   142,  1358,  1301, 12223,  1260,  1884, 26875,     0,     0],
        [  101,  2896,  1161,  1301,  8419,   182, 10950,  1186,  1260,  1435],
        [  101,  1197, 13473,     0,     0,     0,     0,     0,     0,     0]])
tensor([[  142,  1358,  1301, 12223,  1260,  1884, 26875,     0,     0,     0],
        [ 2896,  1161,  1301,  8419,   182, 10950,  1186,  1260,  1435,     0],
        [ 1197, 13473,     0,     0,     0,     0,     0,     0,     0,     0]])
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset BookCorpus para treinar e avaliar nosso modelo de linguagem.

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!mkdir -p ./downloads
!wget -nc --directory-prefix=downloads https://storage.googleapis.com/huggingface-nlp/datasets/bookcorpus/bookcorpus.tar.bz2

--2022-07-18 19:43:40--  https://storage.googleapis.com/huggingface-nlp/datasets/bookcorpus/bookcorpus.tar.bz2
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 74.125.204.128, 64.233.187.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1179510242 (1.1G) [application/x-bzip2]
Saving to: ‘downloads/bookcorpus.tar.bz2’

bookcorpus.tar.bz2  100%[===================>]   1.10G  64.4MB/s    in 25s     

2022-07-18 19:44:07 (45.7 MB/s) - ‘downloads/bookcorpus.tar.bz2’ saved [1179510242/1179510242]



In [11]:
!mkdir -p ./data
!tar -xf downloads/bookcorpus.tar.bz2 -C data/

In [12]:
!rm downloads/*.tar.bz2

In [14]:
# Load datasets
max_seq_length = 12

# data set pequeno
train_examples = 400000
valid_examples = 800
test_examples = 800

path_text = f'./data/books_large_p1.txt'
texts = open(path_text).readlines()

n_concat=10

print(f'Read {len(texts)//n_concat} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
max_lin = max_lines * n_concat
texts = texts[:max_lin]  
text_aux = ''
text_all = []
j=0
for i,text in enumerate(texts):
    text_aux = f'{text_aux}{text.strip()} '
    if (i+1)%n_concat==0:
        text_all.append(text_aux)
        j=j+1
        text_aux=''

texts = text_all 

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 4000000 lines.
Truncating to 401600 lines.


In [15]:
texts[0]

'the half-ling book one in the fall of igneeria series kaylee soderburg copyright 2013 kaylee soderburg all rights reserved . isbn : 1492913731 isbn-13 : 978-1492913733 for my family , who encouraged me to never stop fighting for my dreams chapter 1 summer vacations supposed to be fun , right ? i wish i had a better answer to that question . starlings , new york is not the place youd expect much to happen . its a small quiet town , the kind where everyone knows your name . its a place where your parents wouldnt even care if you stayed out late biking with your friends . only because everyone felt so safe , so comfy . they dont know the half of it . but i do . i know it all and starlings is not the place where you want to be after dark . '

In [16]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 5887656
valid examples: 10626
test examples: 14891


In [17]:
class MultiheadAttention(torch.nn.Module):
    def __init__(self, dim, n_head):
        """
        Implements the Multi head Attention

        Args:
            dim : Dimension of the embedding layer for each word in the context.
            n_layers : number of self-attention layers.
        """
        super(MultiheadAttention, self).__init__()
        
        self.dim = dim
        self.n_head = n_head

        # Dimension de cada head
        self.dim_head = self.dim // self.n_head

        # As matrixes W_k, W_q, W_v, W_e   
        self.W_k = nn.Linear(self.dim, self.dim , bias=False)
        self.W_q = nn.Linear(self.dim, self.dim , bias=False)
        self.W_v = nn.Linear(self.dim, self.dim , bias=False)
        self.W_e = nn.Linear(self.dim, self.dim , bias=False)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, key, value, query, mask = None):
        """
        Args:
            x is a LongTensor of shape (batch_size, max_seq_length, dimension)  (B,L,D)
            mask is Tensor of shape (batch_size, 1, max_seq_length,max_seq_length) (B,1,L,L)
        Returns:
            LongTensor of shape (batch_size, max_seq_length, dimension)
        """
        batch_size = value.size(0)
        max_seq_length = value.size(1)

        # k, q, v tem dimensão B, L, H, D/H 
        k = self.W_k(key).reshape(batch_size, max_seq_length, self.n_head, self.dim_head)
        q = self.W_q(query).reshape(batch_size, max_seq_length, self.n_head, self.dim_head)
        v = self.W_v(value).reshape(batch_size, max_seq_length, self.n_head, self.dim_head)

        # Transpor para B, H, L, D/H
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)

        # atention
        scores = torch.matmul(q, torch.transpose(k,-1,-2))   # B, H, L, L

        # aplicar mascara
        if mask is not None:
            scores = scores.masked_fill(~mask, float("-1e16"))

        scores = scores/math.sqrt(self.dim_head)

        # aplicar sofmax
        probs = self.softmax(scores) # B, H, L, L

        probs  = torch.matmul(probs, v) # B, H, L, D/H 

        probs = probs.transpose(1,2).contiguous() # B, L, H, D/H 
        probs = probs.reshape(batch_size,max_seq_length, self.dim) # B, L, D

        return self.W_e(probs)



In [18]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, dim: int, n_head: int, expansion_factor: int):
        """
        Implements Transformer Block 
        Args:
            dim (int): Dimension of the embedding layer for each word in the context.
            n_head (int): number of self-attention head
        """
        super(TransformerBlock,self).__init__()

        self.dim = dim
        self.n_head = n_head
        self.expansion_factor = expansion_factor

        self.multi_head = MultiheadAttention(self.dim, self.n_head)

        self.norm1 = nn.LayerNorm(self.dim)
        self.dropout1 = nn.Dropout(0.1)

        hidden_size = self.expansion_factor * self.dim
        self.feed_forward = nn.Sequential(
            nn.Linear(self.dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, self.dim)
        )

        self.norm2 = nn.LayerNorm(self.dim)
        self.dropout2 = nn.Dropout(0.1)
        
    def forward(self, key, value, query, mask=None):
        """
        Args:
            x is a LongTensor of shape (batch_size, max_seq_length, dimension)  (B,L,D)
            mask is Tensor of shape (batch_size, 1, max_seq_length,max_seq_length) (B,1,L,L)
        Returns:
            LongTensor of shape (batch_size, max_seq_length, dimension)
        """

        attention = self.multi_head(key, value, query, mask=mask)
        attention_residual = attention + query
        norm1_out = self.dropout1(self.norm1(attention_residual))
        
        feed_fwd = self.feed_forward(norm1_out)
        feed_fwd_residual = feed_fwd + norm1_out
        norm2_out = self.dropout2(self.norm2(feed_fwd_residual))
        
        return norm2_out


In [20]:
class DecoderBlock(torch.nn.Module):
    def __init__(self, dim: int, n_head: int, expansion_factor: int):

        super(DecoderBlock, self).__init__()

        self.dim = dim
        self.n_head = n_head
        self.expasion_factor = expansion_factor

        self.attention = MultiheadAttention(self.dim, self.n_head)
        self.norm1 = nn.LayerNorm(self.dim)
        self.dropout1 = nn.Dropout(0.1)

        self.transformer_block = TransformerBlock(self.dim, self.n_head, self.expasion_factor)
    
    def forward(self, enc_out, x, mask_self,mask_cross=None):

        attention = self.attention(x, x, x, mask_self)
        attention_residual = attention + x
        out = self.dropout1(self.norm1(attention_residual))

        if enc_out is None:
            enc_out = out
        else:
            mask_cross = None
        out = self.transformer_block(enc_out,enc_out,out,mask_cross)

        return out

In [21]:
class TransformerDecoder(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int, n_head: int, expansion_factor: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
            n_head(int): number of head of self-attention
            expansion_factor(int): fator for the hidden size of feed_forward
        """
        # Escreva seu código aqui.
        super(TransformerDecoder,self).__init__()
        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length
        self.dim = dim
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id

        self.n_head = n_head
        self.expansion_factor = expansion_factor

        # C()
        self.C_w = nn.Embedding(vocab_size, dim)

        # P()
        self.P_w = nn.Embedding(max_seq_length, dim)

        self.dropout = nn.Dropout(0.1)

        self.layers = nn.ModuleList([DecoderBlock(self.dim,self.n_head,self.expansion_factor) for i in range(self.n_layers)])

        self.linear_out = nn.Linear(self.dim,self.vocab_size)

        
    def forward(self, inputs, enc_out=None):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        # Escreva seu código aqui.
        
        batch_size = inputs.size(0)
        max_seq_length = inputs.size(1)

        c_emb = self.C_w(inputs)  # B,L,D
        p_emb = self.P_w(torch.LongTensor(range(0,self.max_seq_length)).unsqueeze(0).to(inputs.device))


        x = c_emb + p_emb  # B,L,D
        x = self.dropout(x)

        # generar mascara
        mask_tri = (torch.ones(self.max_seq_length,self.max_seq_length).tril() == 1).expand(batch_size,1,self.max_seq_length,self.max_seq_length).to(inputs.device)
        
        mask_pad = (inputs != self.pad_token_id)
        mask_pad = mask_pad.reshape(batch_size,1,1,self.max_seq_length).expand(batch_size,1,self.max_seq_length,self.max_seq_length).to(inputs.device)

        mask = torch.logical_and(mask_tri,mask_pad)
        mask_cross = mask_tri
        for layer in self.layers:

            x = layer(enc_out,x,mask_self=mask,mask_cross=mask_cross)   # B, L, D

        out = self.linear_out(x) # B, L, vocab

        return out

## Teste o modelo com um exemplo

In [22]:
model = TransformerDecoder(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=256,
    n_layers=4,
    pad_token_id=tokenizer.pad_token_id,
    n_head = 4,
    expansion_factor = 4,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 12])
sample_output.shape: torch.Size([1, 12, 28996])


In [23]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 19083588


## Assert da Perplexidade


In [24]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              34315
correct initial perplexity: 28996
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [25]:
model_save_name = 'model_decoder_Bookcorpus.pt'
path_modelo = F"/content/drive/My Drive/modelo_proyecto/entrega_03/pre_trained/{model_save_name}"

In [27]:
max_examples = 150_000_000
eval_every_steps = 1000
lr = 3e-4


dim = 256
n_layers = 4
n_head = 4
expansion_factor = 4


model = TransformerDecoder(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=dim,
    n_layers=n_layers,
    pad_token_id=tokenizer.pad_token_id,
    n_head = n_head,
    expansion_factor = expansion_factor,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

checkpoint = {'vocab_size': tokenizer.vocab_size,
              'max_seq_length': max_seq_length,
              'dim': dim,
              'n_layers': n_layers,
              'pad_token_id': tokenizer.pad_token_id,
              'n_head': n_head,
              'expansion_factor': expansion_factor,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict' : optimizer.state_dict(),
              'step': [],
              'n_examples': [],
              'train_ppl': [],
              'valid_ppl': []
}

torch.save(checkpoint, path_modelo)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)
            train_ppl = np.exp(train_loss)

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train loss: {train_loss:.6f}, train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []
            
            checkpoint['step'].append(step)
            checkpoint['n_examples'].append(n_examples)
            checkpoint['train_ppl'].append(train_ppl)
            checkpoint['valid_ppl'].append(valid_ppl)

            checkpoint['model_state_dict'] = model.state_dict()
            checkpoint['optimizer_state_dict'] = optimizer.state_dict()

            torch.save(checkpoint, path_modelo)

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train loss: 10.437556, train ppl: 34117.18, valid ppl: 24231.57
1000 steps; 64000 examples so far; train loss: 6.836864, train ppl: 931.56, valid ppl: 766.26
2000 steps; 128000 examples so far; train loss: 6.358612, train ppl: 577.44, valid ppl: 421.02
3000 steps; 192000 examples so far; train loss: 5.967823, train ppl: 390.65, valid ppl: 349.89
4000 steps; 256000 examples so far; train loss: 5.802647, train ppl: 331.18, valid ppl: 318.71
5000 steps; 320000 examples so far; train loss: 5.719031, train ppl: 304.61, valid ppl: 301.08
6000 steps; 384000 examples so far; train loss: 5.652295, train ppl: 284.94, valid ppl: 284.15
7000 steps; 448000 examples so far; train loss: 5.585078, train ppl: 266.42, valid ppl: 271.39
8000 steps; 512000 examples so far; train loss: 5.531622, train ppl: 252.55, valid ppl: 264.44
9000 steps; 576000 examples so far; train loss: 5.502714, train ppl: 245.36, valid ppl: 254.64
10000 steps; 640000 examples so far; train loss: 5.463

KeyboardInterrupt: ignored

In [32]:
# seguir entrenando modelo guardado
max_examples = 150_000_000
eval_every_steps = 5000
lr = 2.5e-4

checkpoint = torch.load(path_modelo)
model = TransformerDecoder(
    vocab_size = checkpoint['vocab_size'],
    max_seq_length = checkpoint['max_seq_length'],
    dim = checkpoint['dim'],
    n_layers = checkpoint['n_layers'],
    pad_token_id = checkpoint['pad_token_id'],
    n_head = checkpoint['n_head'],
    expansion_factor = checkpoint['expansion_factor'],
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []

for s in zip(checkpoint['step'], checkpoint['n_examples'], checkpoint['train_ppl'], checkpoint['valid_ppl']):
    print(f'{s[0]} steps; {s[1]} examples so far; train ppl: {s[2]:.2f}, valid ppl: {s[3]:.2f}')

n_examples = checkpoint['n_examples'][-1]
step = checkpoint['step'][-1]

while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_loss = np.average(train_losses)
            train_ppl = np.exp(train_loss)

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train loss: {train_loss:.6f}, train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []
            
            checkpoint['step'].append(step)
            checkpoint['n_examples'].append(n_examples)
            checkpoint['train_ppl'].append(train_ppl)
            checkpoint['valid_ppl'].append(valid_ppl)

            checkpoint['model_state_dict'] = model.state_dict()
            checkpoint['optimizer_state_dict'] = optimizer.state_dict()

            torch.save(checkpoint, path_modelo)

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 34117.18, valid ppl: 24231.57
64000 steps; 64000 examples so far; train ppl: 931.56, valid ppl: 766.26
128000 steps; 128000 examples so far; train ppl: 577.44, valid ppl: 421.02
192000 steps; 192000 examples so far; train ppl: 390.65, valid ppl: 349.89
256000 steps; 256000 examples so far; train ppl: 331.18, valid ppl: 318.71
320000 steps; 320000 examples so far; train ppl: 304.61, valid ppl: 301.08
384000 steps; 384000 examples so far; train ppl: 284.94, valid ppl: 284.15
448000 steps; 448000 examples so far; train ppl: 266.42, valid ppl: 271.39
512000 steps; 512000 examples so far; train ppl: 252.55, valid ppl: 264.44
576000 steps; 576000 examples so far; train ppl: 245.36, valid ppl: 254.64
640000 steps; 640000 examples so far; train ppl: 235.93, valid ppl: 248.29
704000 steps; 704000 examples so far; train ppl: 229.78, valid ppl: 245.02
768000 steps; 768000 examples so far; train ppl: 223.95, valid ppl: 240.36
832000 steps; 832000 examples so 

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [33]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 161.88555274217737


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [34]:
prompt = 'A man is eating pizza in the park at night with a'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A man is eating pizza in the park at night with a copyright
A man is eating pizza in the park at night with a copyright 2014
A man is eating pizza in the park at night with a copyright 2014 :
A man is eating pizza in the park at night with a copyright 2014 : /
A man is eating pizza in the park at night with a copyright 2014 : / /
A man is eating pizza in the park at night with a copyright 2014 : / / www
A man is eating pizza in the park at night with a copyright 2014 : / / www.
A man is eating pizza in the park at night with a copyright 2014 : / / www. /
A man is eating pizza in the park at night with a copyright 2014 : / / www. / /
A man is eating pizza in the park at night with a copyright 2014 : / / www. / / /
A man is eating pizza in the park at night with a copyright 2014 : / / www. / / / /
A man is eating pizza in the park at night with a copyright 2014 : / / www. / / / / /
A man is eating pizza in the park at night with a copyright 2014 : / / www. / / / / / /
A man is eating piz

In [35]:
prompt = 'The young dog is sleeping in the park and playing with an'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

The young dog is sleeping in the park and playing with an hour
The young dog is sleeping in the park and playing with an hour.
The young dog is sleeping in the park and playing with an hour. to
The young dog is sleeping in the park and playing with an hour. to the
The young dog is sleeping in the park and playing with an hour. to the in
The young dog is sleeping in the park and playing with an hour. to the india
The young dog is sleeping in the park and playing with an hour. to the india,
The young dog is sleeping in the park and playing with an hour. to the india, the
The young dog is sleeping in the park and playing with an hour. to the india, the united
The young dog is sleeping in the park and playing with an hour. to the india, the united states
The young dog is sleeping in the park and playing with an hour. to the india, the united states had
The young dog is sleeping in the park and playing with an hour. to the india, the united states had been
The young dog is sleeping in the p

In [36]:
prompt = 'The old dog is running in the park and playing with a'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

The old dog is running in the park and playing with a new
The old dog is running in the park and playing with a new yo
The old dog is running in the park and playing with a new york
The old dog is running in the park and playing with a new york.
The old dog is running in the park and playing with a new york..
The old dog is running in the park and playing with a new york.. south
The old dog is running in the park and playing with a new york.. south,
The old dog is running in the park and playing with a new york.. south, chapter
The old dog is running in the park and playing with a new york.. south, chapter 2
The old dog is running in the park and playing with a new york.. south, chapter 2 :
The old dog is running in the park and playing with a new york.. south, chapter 2 : http
The old dog is running in the park and playing with a new york.. south, chapter 2 : http :
The old dog is running in the park and playing with a new york.. south, chapter 2 : http : /
The old dog is running in t

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.